# Analysis of Runtime and Solutions

In [ ]:
import os
import json
import pathlib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import ScalarFormatter,AutoMinorLocator
import matplotlib as mpl
from pprint import pprint

In [ ]:
color_wheel = ['#456990', '#EBA947', '#76E5C4', '#EF1A2C', '#fddbc7', '#f7f7f7', '#d1e5f0', '#92c5de', '#4393c3']

In [ ]:
project_path = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
report_path = f'{project_path}/report/'
machine = 'solstorm'  # solstorm or local
dir_names = ['base_case', 'fleet_size', 'order_composition', 'selection_strategy', 'weather', 'no_speed_opt']

dir_name_to_files = {}
for dir_name in dir_names:
    results_path = f'{project_path}/output/{machine}/{dir_name}/results'

    files = [os.path.join(results_path, f) for f in os.listdir(results_path) 
             if os.path.isfile(os.path.join(results_path, f))]
    
    dir_name_to_files.update({dir_name: files})

In [ ]:
def count_installations(results):
    installations = set()
    for order in results['instance_info']['order_composition']:
        installations.add(results['instance_info']['order_composition'][order]['installation'])
    return len(installations)

def make_runtime_df(files, sortby):
    columns = ['Instance name', 'Orders', 'Vessels', 'Installations',
               'Weather', 'Selection', 'Optimality gap', 'Variables', 'Arcs', 
               'Preprocess runtime', 'Model runtime']
    rows = []
    for file in files:
        file_name = os.path.basename(file).split('.')[0]
        with open(file) as file:
            results = json.load(file)
        orders = len(results['order_fulfillment']['postponed_orders']) + len(results['order_fulfillment']['serviced_orders'])
        installations = count_installations(results)
        category = file_name.split('-')[1]
        selection = 'Clustered' if category == 'CL' else 'Even' if category == 'ES' else 'Random'
        row = [file_name,
               orders,
               results['instance_info']['fleet_size'],
               installations,
               results['instance_info']['weather_scenario']+1,
               selection,
               results['objective']['optimality_gap'],
               results['variables']['number_of_variables'],
               results['variables']['number_of_arcs'],
               results['runtime']['preprocess_runtime'],
               results['runtime']['model_runtime']]
        rows.append(row)
    runtime_df = pd.DataFrame(rows, columns=columns)
    runtime_df = runtime_df.sort_values(sortby)
    runtime_df.reset_index(drop=True, inplace=True)
    return runtime_df

# pd.set_option("display.max_rows", None, "display.max_columns", None)

bc_runtime_df = make_runtime_df(dir_name_to_files['base_case'], ['Orders'])
fs_runtime_df = make_runtime_df(dir_name_to_files['fleet_size'], ['Vessels', 'Orders', 'Instance name'])
fs_runtime_small_df = fs_runtime_df[fs_runtime_df['Orders'].isin([4, 6, 8])]
fs_runtime_medium_df = fs_runtime_df[fs_runtime_df['Orders'].isin([10, 12, 14])]
oc_runtime_df = make_runtime_df(dir_name_to_files['order_composition'], ['Orders','Instance name'])
ss_runtime_df = make_runtime_df(dir_name_to_files['selection_strategy'], ['Orders','Instance name'])
ws_runtime_df = make_runtime_df(dir_name_to_files['weather'], ['Weather', 'Orders','Instance name'])
nso_runtime_df = make_runtime_df(dir_name_to_files['no_speed_opt'], ['Orders', 'Instance name'])

nso_runtime_df

In [ ]:
def mean(x):
    return sum(x) / len(x)

def aggregate_runtime_df(df, groupby):
    agg_runtime_df = df.groupby(groupby).agg(gap=pd.NamedAgg(column='Optimality gap', 
                                                             aggfunc=mean),
                                             variables=pd.NamedAgg(column='Variables',
                                                                   aggfunc=mean),
                                             arcs=pd.NamedAgg(column='Arcs',
                                                              aggfunc=mean),
                                             preprocess_runtime=pd.NamedAgg(column='Preprocess runtime', 
                                                                            aggfunc=mean),
                                             model_runtime=pd.NamedAgg(column='Model runtime',
                                                                       aggfunc=mean))
    agg_runtime_df.variables = agg_runtime_df.variables.astype(int)
    agg_runtime_df.arcs = agg_runtime_df.arcs.astype(int)
    old_to_new_names = {'gap': 'Optimality gap', 
                        'variables': 'Variables',
                        'arcs': 'Arcs',
                        'preprocess_runtime': 'Preprocess runtime',
                        'model_runtime': 'Model runtime'}
    agg_runtime_df = agg_runtime_df.rename(columns=old_to_new_names)
    return agg_runtime_df

bc_agg_runtime_df = aggregate_runtime_df(bc_runtime_df, ['Orders'])
fs_agg_runtime_small_df = aggregate_runtime_df(fs_runtime_small_df, ['Orders', 'Vessels'])
fs_agg_runtime_medium_df = aggregate_runtime_df(fs_runtime_medium_df, ['Orders', 'Vessels'])
ws2_agg_runtime_df = aggregate_runtime_df(ws_runtime_df.loc[ws_runtime_df['Weather'] == 2], ['Orders'])
ws3_agg_runtime_df = aggregate_runtime_df(ws_runtime_df.loc[ws_runtime_df['Weather'] == 3], ['Orders'])
sse_runtime_df = ss_runtime_df.loc[ss_runtime_df['Selection'] == 'Even']
sse_runtime_df = sse_runtime_df.set_index('Orders')
ssc_runtime_df = ss_runtime_df.loc[ss_runtime_df['Selection'] == 'Clustered']
ssc_runtime_df = ssc_runtime_df.set_index('Orders')
nso_agg_runtime_df = aggregate_runtime_df(nso_runtime_df, ['Orders'])

nso_agg_runtime_df

In [ ]:
def make_solutions_df(files, sortby):
    columns = ['Instance name', 'Orders', 'Serviced', 'Postponed', 
               'Vessels', 'Fleet', 'Chartered', 'Weather', 'Selection',
               'Fuel costs', 'Charter costs', 'Penalty costs', 'Objective bound']
    rows = []
    for file in files:
        file_name = os.path.basename(file).split('.')[0]
        with open(file) as file:
            results = json.load(file)
        orders = len(results['order_fulfillment']['postponed_orders']) + len(results['order_fulfillment']['serviced_orders'])
        selection = 'Clustered' if file_name.split('-')[1] == 'CL' else 'Even' if file_name.split('-')[1] == 'ES' else 'Random'
        row = [file_name,
               orders,
               len(results['order_fulfillment']['serviced_orders']),
               len(results['order_fulfillment']['postponed_orders']),  
               results['instance_info']['fleet_size'],
               results['vessels']['fleet_vessels'], 
               results['vessels']['chartered_vessels'],
               results['instance_info']['weather_scenario']+1,
               selection,
               results['objective']['fuel_costs'], 
               results['objective']['charter_costs'], 
               abs(results['objective']['penalty_costs']),
               results['objective']['objective_bound']]
        rows.append(row)
    solutions_df = pd.DataFrame(rows, columns=columns)
    solutions_df = solutions_df.sort_values(sortby)
    solutions_df.reset_index(drop=True, inplace=True)
    return solutions_df

bc_solutions_df = make_solutions_df(dir_name_to_files['base_case'], ['Orders', 'Instance name'])
fs_solutions_df = make_solutions_df(dir_name_to_files['fleet_size'], ['Orders', 'Vessels', 'Instance name'])
fs_solutions_small_df = fs_solutions_df[fs_solutions_df['Orders'].isin([4, 6, 8])]
fs_solutions_medium_df = fs_solutions_df[fs_solutions_df['Orders'].isin([10, 12, 14])]
oc_solutions_df = make_solutions_df(dir_name_to_files['order_composition'], ['Orders', 'Instance name'])
ss_solutions_df = make_solutions_df(dir_name_to_files['selection_strategy'], ['Orders', 'Instance name'])
ws_solutions_df = make_solutions_df(dir_name_to_files['weather'], ['Orders', 'Instance name'])
nso_solutions_df = make_solutions_df(dir_name_to_files['no_speed_opt'], ['Orders', 'Instance name'])

nso_solutions_df

In [ ]:
def mean(x):
    return sum(x) / len(x)

def aggregate_solutions_df(df, groupby):
    agg_solutions_df = df.groupby(groupby).agg(serviced=pd.NamedAgg(column='Serviced',
                                                                    aggfunc=mean),
                                               postponed=pd.NamedAgg(column='Postponed',
                                                                     aggfunc=mean),
                                               fleet_vessels=pd.NamedAgg(column='Fleet',
                                                                         aggfunc=mean),
                                               charter_vessels=pd.NamedAgg(column='Chartered',
                                                                           aggfunc=mean),
                                               fuel_costs=pd.NamedAgg(column='Fuel costs',
                                                                      aggfunc=mean),
                                               charter_costs=pd.NamedAgg(column='Charter costs',
                                                                         aggfunc=mean),
                                               penalty_costs=pd.NamedAgg(column='Penalty costs',
                                                                         aggfunc=mean),
                                               objective_bound=pd.NamedAgg(column='Objective bound',
                                                                           aggfunc=mean))
    old_to_new_names = {'serviced': 'Serviced',
                      'postponed': 'Postponed',
                      'fleet_vessels': 'Fleet',
                      'charter_vessels': 'Chartered',
                      'fuel_costs': 'Fuel costs',
                      'charter_costs': 'Charter costs',
                      'penalty_costs': 'Penalty costs',
                      'objective_bound': 'Objective bound'}
    agg_solutions_df = agg_solutions_df.rename(columns=old_to_new_names)
    return agg_solutions_df

bc_agg_solutions_df = aggregate_solutions_df(bc_solutions_df, ['Orders'])
fs_agg_solutions_small_df = aggregate_solutions_df(fs_solutions_small_df, ['Vessels'])
fs_agg_solutions_medium_df = aggregate_solutions_df(fs_solutions_medium_df, ['Vessels'])
ws_agg_solutions_df = aggregate_solutions_df(ws_solutions_df, ['Orders', 'Weather'])
ss_agg_solutions_df = aggregate_solutions_df(ss_solutions_df, ['Orders', 'Selection'])
nso_agg_solutions_df = aggregate_solutions_df(nso_solutions_df, ['Orders'])

nso_agg_solutions_df

In [ ]:
def plot_columns(df, columns, x_label, y_label, fig, ax, mode, 
                 super_color=None, os=None, ws=None, ss=None, ls=None, nso=None):    
    linewidth = 1.8
    alpha = 0.9
    
    X, Y, colors, labels = [], [], [], []
    if mode == 'base_case':
        for color_choice, column in enumerate(columns):
            x, y = [], []
            for i, value in enumerate(df[column]):
                x.append(int(df[df == value].index[i]))
                y.append(value)
            X.append(x)
            Y.append(y)
            colors.append(color_choice if not super_color else super_color)
            labels.append(column)
    
    elif mode == 'fleet_size':
        for color_choice, column in enumerate(columns):
            x, y = [], []
            for i, value in enumerate(df[columns[0]]):
                x.append(int(df[df == value].index[i]))
                y.append(value)
            X.append(x)
            Y.append(y)
            colors.append(color_choice if not super_color else super_color)
            runtime = columns[0].split(' ')[1].capitalize()
            labels.append(f'{os} orders')
                
    elif mode == 'order_composition':
        for color_choice, column in enumerate(columns):
            x, y = [], []
            for i, value in enumerate(df[column]):
                x.append(df.loc[i, 'Installations'])
                y.append(value)
            X.append(x)
            Y.append(y)
            colors.append(color_choice)
            labels.append(column)
    
    elif mode == 'weather':
        for color_choice, column in enumerate(columns):
            x, y = [], []
            for i, value in enumerate(df[column]):
                x.append(int(df[df == value].index[i]))
                y.append(value)
            X.append(x)
            Y.append(y)
            colors.append(color_choice if not super_color else super_color)
            labels.append(f'Weather scenario {ws}')
    
    elif mode == 'selection_strategy':
        for color_choice, column in enumerate(columns):
            x, y = [], []
            for i, value in enumerate(df[column]):
                x.append(int(df[df == value].index[i]))
                y.append(value)
            X.append(x)
            Y.append(y)
            colors.append(color_choice if not super_color else super_color)
            labels.append(f'Selection strategy {ss}')
    
    elif mode == 'no_speed_opt':
        for color_choice, column in enumerate(columns):
            x, y = [], []
            for i, value in enumerate(df[column]):
                x.append(int(df[df == value].index[i]))
                y.append(value)
            X.append(x)
            Y.append(y)
            colors.append(color_choice if not super_color else super_color)
            labels.append(nso)
    
    for x, y, c, label in zip(X, Y, colors, labels):
        ax.plot(x,
                y,
                color=color_wheel[c],
                linestyle='-' if not ls else ls,
                lw=linewidth if not ls else linewidth * 0.8,
                label=label,
                alpha=alpha)
    return fig, ax

def plot_sub_dfs(df, columns, x_label, y_label, fig, ax, mode):
    orders, vessels = set(), set()
    for o, v in list(df.index):
        orders.add(o)
        vessels.add(v)
        
    for order in orders:
        if order in [4, 10]:
            color = 0
        elif order in [6, 12]:
            color = 2
        elif order in [8, 14]:
            color = 1
        sub_df = df.loc[order]
        plot_columns(sub_df, columns, x_label, y_label, fig, ax, os=order, mode=mode, super_color=color)
    
    if min(vessels) != 1:
        ax.plot(list(vessels), 
                [3600 for _ in range(len(list(vessels)))],
                color=color_wheel[3],
                linestyle='dashed',
                lw=1.3,
                label='Max runtime',
                alpha=1)
    
        ax.set_xlabel(x_label)
        ax.set_ylabel(y_label)
        ax.yaxis.set_major_formatter(ScalarFormatter())
        ax.yaxis.major.formatter._useMathText = True
        ax.yaxis.set_minor_locator(AutoMinorLocator(5))
        ax.xaxis.set_minor_locator(AutoMinorLocator(5))
        ax.locator_params(integer=True)
        ax.legend(frameon=False, bbox_to_anchor=(0.015, 0.96), loc='upper left', ncol=1, handlelength=4)
        # plt.savefig('Fleet_size_runtime_1.jpg', dpi=400)
    else:    
        ax.set_xlabel(x_label)
        ax.set_ylabel(y_label)
        ax.yaxis.set_major_formatter(ScalarFormatter())
        ax.yaxis.major.formatter._useMathText = True
        ax.yaxis.set_minor_locator(AutoMinorLocator(5))
        ax.xaxis.set_minor_locator(AutoMinorLocator(5))
        ax.locator_params(integer=True)
        ax.legend(frameon=False, loc='best', ncol=1, handlelength=4)
        # plt.savefig('Fleet_size_runtime_2.jpg', dpi=400)
    return fig, ax
    
def plot_bc_runtime():
    plt.close('all')
    plt.style.use(os.path.join(report_path, 'PaperDoubleFig.mplstyle'))
    fig, ax = plt.subplots()
    columns = ['Preprocess runtime', 'Model runtime']
    x_label = 'Orders (#)'
    y_label = 'Runtime (s)'
    fig, ax = plot_columns(bc_agg_runtime_df, columns, x_label, y_label, fig, ax, mode='base_case')
    ax.plot([i for i in range(4, 17, 2)], 
            [3600 for _ in range(4, 17, 2)],
            color=color_wheel[3],
            linestyle='dashed',
            lw=1.3,
            label='Max runtime',
            alpha=1)
    ax.set_xlabel(x_label)
    ax.set_ylabel(y_label)
    ax.yaxis.set_major_formatter(ScalarFormatter())
    ax.yaxis.major.formatter._useMathText = True
    ax.yaxis.set_minor_locator(AutoMinorLocator(5))
    ax.xaxis.set_minor_locator(AutoMinorLocator(5))
    ax.locator_params(integer=True)
    ax.legend(frameon=False, loc='best', ncol=1, handlelength=4)
    # plt.savefig('Base_case_runtime.jpg', dpi=400)
    plt.show()

def plot_fs_runtime(df):
    plt.close('all')
    plt.style.use(os.path.join(report_path, 'PaperDoubleFig.mplstyle'))
    fig, ax = plt.subplots()
    columns = ['Model runtime']
    x_label = 'Vessels (#)'
    y_label = 'Runtime (s)'
    # fig, ax = plot_columns(fs_ag_runtime_selected_df, columns, x_label, y_label, fig, ax, mode='fleet_size')
    fig, ax = plot_sub_dfs(df, columns, x_label, y_label, fig, ax, mode='fleet_size')
    ax.set_xlabel(x_label)
    ax.set_ylabel(y_label)
    ax.yaxis.set_major_formatter(ScalarFormatter())
    ax.yaxis.major.formatter._useMathText = True
    ax.yaxis.set_minor_locator(AutoMinorLocator(5))
    ax.xaxis.set_minor_locator(AutoMinorLocator(5))
    ax.locator_params(integer=True)
    plt.show()
    
def plot_oc_runtime():
    plt.close('all')
    plt.style.use(os.path.join(report_path, 'PaperDoubleFig.mplstyle'))
    fig, ax = plt.subplots()
    columns = ['Model runtime']
    x_label = 'Installations (#)'
    y_label = 'Runtime (s)'
    fig, ax = plot_columns(oc_runtime_df, columns, x_label, y_label, fig, ax, mode='order_composition')
    ax.set_xlabel(x_label)
    ax.set_ylabel(y_label)
    ax.yaxis.set_major_formatter(ScalarFormatter())
    ax.yaxis.major.formatter._useMathText = True
    ax.yaxis.set_minor_locator(AutoMinorLocator(5))
    ax.xaxis.set_minor_locator(AutoMinorLocator(5))
    ax.locator_params(integer=True)
    # plt.savefig('Order_composition.jpg', dpi=400)
    plt.show()
    
def plot_ws_runtime():
    plt.close('all')
    plt.style.use(os.path.join(report_path, 'PaperDoubleFig.mplstyle'))
    fig, ax = plt.subplots()
    columns = ['Model runtime']
    x_label = 'Orders (#)'
    y_label = 'Runtime (s)'
    fig, ax = plot_columns(bc_agg_runtime_df, columns, x_label, y_label, fig, ax, 
                           mode='weather', super_color=0, ws='None')
    fig, ax = plot_columns(ws2_agg_runtime_df, columns, x_label, y_label, fig, ax, 
                           mode='weather', super_color=1, ws='Low', ls='dashdot')
    fig, ax = plot_columns(ws3_agg_runtime_df, columns, x_label, y_label, fig, ax, 
                           mode='weather', super_color=2, ws='High')
    ax.plot([i for i in range(4, 27, 2)], 
            [3600 for _ in range(4, 27, 2)],
            color=color_wheel[3],
            linestyle='dashed',
            lw=1.3,
            label='Max runtime',
            alpha=1)
    ax.set_xlabel(x_label)
    ax.set_ylabel(y_label)
    ax.yaxis.set_major_formatter(ScalarFormatter())
    ax.yaxis.major.formatter._useMathText = True
    ax.yaxis.set_minor_locator(AutoMinorLocator(5))
    ax.xaxis.set_minor_locator(AutoMinorLocator(5))
    ax.locator_params(integer=True)
    ax.legend(frameon=False, loc='best', ncol=1, handlelength=4)
    # plt.savefig('Weather_scenarios_runtime.jpg', dpi=400)
    plt.show()
    
def plot_ss_runtime():
    plt.close('all')
    plt.style.use(os.path.join(report_path, 'PaperDoubleFig.mplstyle'))
    fig, ax = plt.subplots()
    columns = ['Model runtime']
    x_label = 'Orders (#)'
    y_label = 'Runtime (s)'
    fig, ax = plot_columns(bc_agg_runtime_df, columns, x_label, y_label, fig, ax, 
                           mode='selection_strategy', super_color=0, ss='Random')
    fig, ax = plot_columns(ssc_runtime_df, columns, x_label, y_label, fig, ax, 
                           mode='selection_strategy', super_color=1, ss='Clustering')
    fig, ax = plot_columns(sse_runtime_df, columns, x_label, y_label, fig, ax, 
                           mode='selection_strategy', super_color=2, ss='Even spreading')
    ax.plot([i for i in range(4, 19, 2)], 
            [3600 for _ in range(4, 19, 2)],
            color=color_wheel[3],
            linestyle='dashed',
            lw=1.3,
            label='Max runtime',
            alpha=1)
    ax.set_xlabel(x_label)
    ax.set_ylabel(y_label)
    ax.yaxis.set_major_formatter(ScalarFormatter())
    ax.yaxis.major.formatter._useMathText = True
    ax.yaxis.set_minor_locator(AutoMinorLocator(5))
    ax.xaxis.set_minor_locator(AutoMinorLocator(5))
    ax.locator_params(integer=True)
    ax.legend(frameon=False, loc='best', ncol=1, handlelength=4)
    # plt.savefig('Selection_strategy_runtime.jpg', dpi=400)
    plt.show()
    
def plot_nso_runtime():
    plt.close('all')
    plt.style.use(os.path.join(report_path, 'PaperDoubleFig.mplstyle'))
    fig, ax = plt.subplots()
    columns = ['Model runtime']
    x_label = 'Orders (#)'
    y_label = 'Runtime (s)'
    fig, ax = plot_columns(bc_agg_runtime_df, columns, x_label, y_label, fig, ax, 
                           mode='no_speed_opt', super_color=1, nso='Speed optimization')
    fig, ax = plot_columns(nso_agg_runtime_df, columns, x_label, y_label, fig, ax,
                           mode='no_speed_opt', super_color=0, nso='No speed optimization')
    ax.plot([i for i in range(4, 21, 2)], 
            [3600 for _ in range(4, 21, 2)],
            color=color_wheel[3],
            linestyle='dashed',
            lw=1.3,
            label='Max runtime',
            alpha=1)
    ax.set_xlabel(x_label)
    ax.set_ylabel(y_label)
    ax.yaxis.set_major_formatter(ScalarFormatter())
    ax.yaxis.major.formatter._useMathText = True
    ax.yaxis.set_minor_locator(AutoMinorLocator(5))
    ax.xaxis.set_minor_locator(AutoMinorLocator(5))
    ax.locator_params(integer=True)
    ax.legend(frameon=False, loc='best', ncol=1, handlelength=4)
    # plt.savefig('No_speed_opt_runtime.jpg', dpi=400)
    plt.show()

## Base case

### Base case runtime

In [ ]:
bc_runtime_df

#### Base case runtime aggregated

In [ ]:
bc_agg_runtime_df

In [ ]:
plot_bc_runtime()

### Base case solutions

In [ ]:
bc_solutions_df

#### Base case solutions aggregated

In [ ]:
bc_agg_solutions_df

## Fleet size

### Fleet size runtime

In [ ]:
fs_runtime_df

#### Fleet size runtime aggregated

In [ ]:
fs_agg_runtime_small_df

In [ ]:
plot_fs_runtime(fs_agg_runtime_small_df)

In [ ]:
fs_agg_runtime_medium_df

In [ ]:
plot_fs_runtime(fs_agg_runtime_medium_df)

### Fleet size solutions

In [ ]:
fs_solutions_df

#### Fleet size solutions aggregated

In [ ]:
fs_agg_solutions_small_df

In [ ]:
fs_agg_solutions_medium_df

## Order composition

### Order composition runtime

In [ ]:
oc_runtime_df

In [ ]:
plot_oc_runtime()

### Order composition solutions

In [ ]:
oc_solutions_df

## Selection strategy

### Selection strategy runtime

In [ ]:
ss_runtime_df

#### Selection strategy Even Spreading

In [ ]:
sse_runtime_df

#### Selection strategy Clustering

In [ ]:
ssc_runtime_df

### Selection strategy solutions

In [ ]:
ss_solutions_df

#### Selection strategy solutions aggregated

In [ ]:
ss_agg_solutions_df

In [ ]:
plot_ss_runtime()

## Weather scenario

### Weather scenario runtime

In [ ]:
ws_runtime_df

#### Weather scenario Low aggregated runtime

In [ ]:
ws2_agg_runtime_df

#### Weather scenario High aggregated runtime

In [ ]:
ws3_agg_runtime_df

In [ ]:
plot_ws_runtime()

### Weather scenario solutions

In [ ]:
ws_solutions_df

#### Weather scenario aggregated solutions

In [ ]:
ws_agg_solutions_df

## Speed optimization

### Speed optimization runtime

In [ ]:
nso_runtime_df

In [ ]:
nso_agg_runtime_df

In [ ]:
plot_nso_runtime()